<a href="https://colab.research.google.com/github/ZaneZaiontz/YosAI/blob/main/YosAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Temp
